In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

### Read the Data

In [2]:
raw_data = pd.read_excel(r"C:\zubeda\ML Assignement\Online Retail.xlsx")
raw_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Understanding and Pre-processing

In [3]:
raw_data.shape

(541909, 8)

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
raw_data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
user_data = raw_data.dropna()
user_data.shape

(406829, 8)

In [7]:
user_data.Country.value_counts().reset_index()

,index,Country
0,United Kingdom,361878
1,Germany,9495
2,France,8491
3,EIRE,7485
4,Spain,2533
5,Netherlands,2371
6,Belgium,2069
7,Switzerland,1877
8,Portugal,1480
9,Australia,1259


In [8]:
user_data.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


**Creating Amount Feature**

In [9]:
user_data["Amount"] = user_data.Quantity * user_data.UnitPrice

#remove negative amount
user_data = user_data[user_data["Amount"] > 0]

user_data.shape

(397884, 9)

In [10]:
user_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [11]:
user_data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
Amount         0
dtype: int64

### Create RFM model (Recency, Frequency, Monetary)

**Create customer Recency feature**

In [12]:
import datetime

#reference date as one day before the last transaction
ref_date = user_data.InvoiceDate.max()
ref_date = ref_date + datetime.timedelta(days=1)     #timedelta function returns to total number of seconds
ref_date

Timestamp('2011-12-10 12:50:00')

In [13]:
user_data['days_last_purchase'] = ref_date - user_data.InvoiceDate
user_data['days_last_purchase_num'] = user_data['days_last_purchase'].astype('timedelta64[D]')
user_data['days_last_purchase_num'].sample(5)  

380869     64.0
444932     33.0
178282    217.0
481909     21.0
149395    246.0
Name: days_last_purchase_num, dtype: float64

In [14]:
#Customer transaction history
customer_hist_df = pd.DataFrame(user_data.groupby("CustomerID")["days_last_purchase_num"].min()).reset_index()
customer_hist_df.rename(columns={"days_last_purchase_num": "Recency"}, inplace=True)

customer_hist_df.head()

,CustomerID,Recency
0,12346.0,326.0
1,12347.0,2.0
2,12348.0,75.0
3,12349.0,19.0
4,12350.0,310.0


**Create Frequency and Monetary (Amount) feature**

In [15]:
customer_monetary = user_data[["CustomerID","Amount"]].groupby('CustomerID')["Amount"].sum().reset_index()
customer_hist_df = customer_hist_df.merge(customer_monetary, how='outer')

customer_freq = user_data[['CustomerID','Amount']].groupby('CustomerID')["Amount"].count().reset_index()
customer_freq.rename(columns={'Amount': 'Frequency'}, inplace=True)
customer_hist_df = customer_hist_df.merge(customer_freq, how='outer')

customer_hist_df = pd.DataFrame(customer_hist_df, columns=['CustomerID','Recency','Amount','Frequency'])
customer_hist_df.head()

,CustomerID,Recency,Amount,Frequency
0,12346.0,326.0,77183.60,1
1,12347.0,2.0,4310.00,182
2,12348.0,75.0,1797.24,31
3,12349.0,19.0,1757.55,73
4,12350.0,310.0,334.40,17


In [16]:
from sklearn import preprocessing

customer_hist_df['Recency_log'] = customer_hist_df["Recency"].apply(np.log)
customer_hist_df['Frequency_log'] = customer_hist_df["Frequency"].apply(np.log)
customer_hist_df['Amount_log'] = customer_hist_df["Amount"].apply(np.log)
features = ["Recency_log","Frequency_log","Amount_log"]

X = pd.DataFrame(customer_hist_df[features].values, columns=features)
X.head()

,Recency_log,Frequency_log,Amount_log
0,5.786897,0.000000,11.253942
1,0.693147,5.204007,8.368693
2,4.317488,3.433987,7.494007
3,2.944439,4.290459,7.471676
4,5.736572,2.833213,5.812338


### KMeans Clustering

In [17]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

n_clusters = [2, 3, 4, 5, 6]

for k in n_clusters:
    kmeans = KMeans(n_clusters=k, random_state=112)
    predict = kmeans.fit_predict(X)
    score = silhouette_score(X, predict, random_state=1)
    print("k = {}, Score = {}".format(k, score))

k = 2, Score = 0.3945178445812027
k = 3, Score = 0.3023683059862129
k = 4, Score = 0.30625195716981807
k = 5, Score = 0.2807436175747306
k = 6, Score = 0.2822786330537753


In [18]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)

KMeans(n_clusters=2)

In [19]:
customer_hist_df1 = customer_hist_df.set_index("CustomerID")[["Recency_log", "Frequency_log", "Amount_log"]]
customer_hist_df1["Cluster"] = kmeans.labels_
customer_hist_df1.sample(10)

,Recency_log,Frequency_log,Amount_log,Cluster
CustomerID,,,,
16403.0,5.743003,2.302585,5.178971,1
12836.0,4.077537,5.164786,7.868201,0
16723.0,4.510860,3.761200,6.378681,1
16235.0,3.713572,3.828641,6.765396,1
16875.0,4.204693,4.718499,7.647562,0
17120.0,5.451038,2.890372,5.728345,1
17640.0,4.465908,1.609438,6.432393,1
14601.0,2.397895,1.945910,5.365789,1
13171.0,3.091042,3.828641,6.149344,0


In [20]:
test = pd.DataFrame(customer_hist_df1.loc[13339.0, features]).T
print("Actual:")
print(customer_hist_df1.loc[13339.0, "Cluster"])
predict = kmeans.predict(test)
print("Predicted:")
predict[0]

Actual:
1
Predicted:


1

In [21]:
y = pd.DataFrame(user_data.groupby('CustomerID')["Quantity"].sum())
y

,Quantity
CustomerID,
12346.0,74215
12347.0,2458
12348.0,2341
12349.0,631
12350.0,197
...,...
18280.0,45
18281.0,54
18282.0,103


In [22]:
y["Quantity_log"] = np.log(y["Quantity"])
y

,Quantity,Quantity_log
CustomerID,,
12346.0,74215,11.214722
12347.0,2458,7.807103
12348.0,2341,7.758333
12349.0,631,6.447306
12350.0,197,5.283204
...,...,...
18280.0,45,3.806662
18281.0,54,3.988984
18282.0,103,4.634729


### Linear Regression

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y["Quantity_log"], random_state = 100)

In [24]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [25]:
predicted = lr_model.predict(X_test)
result_df = pd.DataFrame(columns=["Predicted_quantity", "Actual_quantity"])
result_df["Predicted_quantity"] = predicted
result_df["Actual_quantity"] = y_test.values
result_df.index = y_test.index
result_df

,Predicted_quantity,Actual_quantity
CustomerID,,
16422.0,9.719878,10.425372
15730.0,6.420626,6.535241
14607.0,8.871106,9.372969
12789.0,3.822512,1.386294
15004.0,6.599926,6.445720
...,...,...
16713.0,8.365572,8.360773
16892.0,5.931934,5.476464
13213.0,6.728746,7.215240


In [26]:
from sklearn.metrics import r2_score, mean_squared_error
import math

rmse = math.sqrt(mean_squared_error(y_test, predicted))
r2 = r2_score(y_test, predicted)

print('\nRMSE:', rmse)
print('R2 Score:', r2)


RMSE: 0.5605270208477203
R2 Score: 0.8392596998327324


In [27]:
desc_count = np.arange(1, len(pd.unique(user_data["Description"])) + 1)
desc = pd.unique(user_data["Description"])
desc_dict = dict(zip(desc, desc_count))
desc_dict

{'WHITE HANGING HEART T-LIGHT HOLDER': 1,
 'WHITE METAL LANTERN': 2,
 'CREAM CUPID HEARTS COAT HANGER': 3,
 'KNITTED UNION FLAG HOT WATER BOTTLE': 4,
 'RED WOOLLY HOTTIE WHITE HEART.': 5,
 'SET 7 BABUSHKA NESTING BOXES': 6,
 'GLASS STAR FROSTED T-LIGHT HOLDER': 7,
 'HAND WARMER UNION JACK': 8,
 'HAND WARMER RED POLKA DOT': 9,
 'ASSORTED COLOUR BIRD ORNAMENT': 10,
 "POPPY'S PLAYHOUSE BEDROOM ": 11,
 "POPPY'S PLAYHOUSE KITCHEN": 12,
 'FELTCRAFT PRINCESS CHARLOTTE DOLL': 13,
 'IVORY KNITTED MUG COSY ': 14,
 'BOX OF 6 ASSORTED COLOUR TEASPOONS': 15,
 'BOX OF VINTAGE JIGSAW BLOCKS ': 16,
 'BOX OF VINTAGE ALPHABET BLOCKS': 17,
 'HOME BUILDING BLOCK WORD': 18,
 'LOVE BUILDING BLOCK WORD': 19,
 'RECIPE BOX WITH METAL HEART': 20,
 'DOORMAT NEW ENGLAND': 21,
 'JAM MAKING SET WITH JARS': 22,
 'RED COAT RACK PARIS FASHION': 23,
 'YELLOW COAT RACK PARIS FASHION': 24,
 'BLUE COAT RACK PARIS FASHION': 25,
 'BATH BUILDING BLOCK WORD': 26,
 'ALARM CLOCK BAKELIKE PINK': 27,
 'ALARM CLOCK BAKELIKE RED ':

In [29]:
user_data["Product_id"] = user_data["Description"]
user_data["Product_id"] = user_data["Product_id"].map(desc_dict)
user_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount,days_last_purchase,days_last_purchase_num,Product_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,374 days 04:24:00,374.0,1
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,374 days 04:24:00,374.0,2
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,374 days 04:24:00,374.0,3
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,374 days 04:24:00,374.0,4
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,374 days 04:24:00,374.0,5
